In [1]:
import numpy as np
import pandas as pd

In [12]:
# prompt: read csv file which is named as diabetes

df = pd.read_csv('diabetes.csv')


In [13]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [14]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [15]:
# prompt: divide data into i/p and o/p as x and y

x = df.drop('Outcome',axis=1)
y = df['Outcome']


In [16]:
x=x.values
y=y.values

In [20]:
x.shape

(768, 8)

In [21]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)

In [26]:
# prompt: divide x_Scaled in traing and testing

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.2,random_state=1)


In [245]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [32]:
model=Sequential()
model.add(Dense(32,activation="relu",input_dim=8))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])


In [34]:
model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
20/20 [==============================] - 0s 11ms/step - loss: 0.4648 - accuracy: 0.7801 - val_loss: 0.4679 - val_accuracy: 0.7857
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4623 - accuracy: 0.7866 - val_loss: 0.4659 - val_accuracy: 0.7857
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4597 - accuracy: 0.7899 - val_loss: 0.4627 - val_accuracy: 0.7987
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4574 - accuracy: 0.7850 - val_loss: 0.4637 - val_accuracy: 0.8052
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4560 - accuracy: 0.7801 - val_loss: 0.4619 - val_accuracy: 0.8117
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4541 - accuracy: 0.7818 - val_loss: 0.4590 - val_accuracy: 0.8117
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4529 - accuracy: 0.7818 - val_loss: 0.4584 - val_accuracy: 0.8052
Epoch 8/10
20/20 [=

In [35]:
#how to select approproate optimise
#no of neurons in the layer
#no of hidenn layers
# all in one model


In [41]:
pip install -U keras-tuner

In [148]:
import kerastuner as kt

In [51]:
def build_model(hp):
  model=Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  opti=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=opti,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [52]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [53]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.5844155550003052

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 05s


In [62]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [57]:
# here in down we are trying to get the full best model which is trained above and
# we no nedd to compile it again


In [63]:
model=tuner.get_best_models(num_models=1)[0]

In [64]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

In [149]:
# for number of neurons in each layer

In [182]:
def build_model2(hp):
  model=Sequential()
  units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(Dense(units=units,activation='elu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [183]:
tuner = kt.RandomSearch(build_model2,objective='val_accuracy',max_trials=5,directory='mydir6',project_name='my_project6')

In [184]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 07s


In [185]:
tuner.get_best_hyperparameters()[0].values

{'units': 64}

In [186]:
# Assuming you have already performed hyperparameter tuning with the tuner
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=2)

# Accessing the values of the second-best hyperparameters
second_best_hyperparameters_values = best_hyperparameters[1].values

print("Second-best hyperparameters:")
print(second_best_hyperparameters_values)


Second-best hyperparameters:
{'units': 320}


In [187]:
# Assuming you have already performed hyperparameter tuning with the tuner
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=5)

# Accessing the values of the top 5 best hyperparameters
for i, hyperparameters in enumerate(best_hyperparameters):
    print(f"Top-{i + 1} Hyperparameters:")
    print(hyperparameters.values)


Top-1 Hyperparameters:
{'units': 64}
Top-2 Hyperparameters:
{'units': 320}
Top-3 Hyperparameters:
{'units': 512}
Top-4 Hyperparameters:
{'units': 352}
Top-5 Hyperparameters:
{'units': 224}


In [192]:
model=tuner.get_best_models(num_models=1)[0]

In [193]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 11ms/step - loss: 0.4949 - accuracy: 0.7622 - val_loss: 0.4814 - val_accuracy: 0.7987
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4831 - accuracy: 0.7638 - val_loss: 0.4760 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4776 - accuracy: 0.7736 - val_loss: 0.4730 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4738 - accuracy: 0.7704 - val_loss: 0.4725 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4717 - accuracy: 0.7687 - val_loss: 0.4726 - val_accuracy: 0.7727
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4712 - accuracy: 0.7704 - val_loss: 0.4739 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4699 - accuracy: 0.7736 - val_loss: 0.4736 - val_accuracy: 0.7792
Epoch 14

In [194]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 641 (2.50 KB)
Trainable params: 641 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# how to sleect numnber of layers

In [200]:
def build_model3(hp):
  model=Sequential()
  model.add(Dense(72,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [201]:
tuner=kt.RandomSearch(build_model3,objective='val_accuracy',max_trials=3,directory='mydirr1',project_name='project1')


In [204]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 01s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 05s


In [205]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [206]:
model=tuner.get_best_models()[0]

In [208]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4450 - accuracy: 0.7818 - val_loss: 0.4571 - val_accuracy: 0.7987
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4454 - accuracy: 0.7866 - val_loss: 0.4589 - val_accuracy: 0.8247
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4452 - accuracy: 0.7801 - val_loss: 0.4590 - val_accuracy: 0.8182
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4448 - accuracy: 0.7785 - val_loss: 0.4631 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4451 - accuracy: 0.7834 - val_loss: 0.4601 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4445 - accuracy: 0.7899 - val_loss: 0.4613 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4440 - accuracy: 0.7785 - val_loss: 0.4570 - val_accuracy: 0.8247
Epoch 14/

In [249]:
def build_model4(hp):
  model=Sequential()

  counter=0
  units=hp.Int('neurons',min_value=8,max_value=128,step=8)
  activation=hp.Choice('activation', values=['relu', 'sigmoid', 'tanh'])
  opti=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  loss=hp.Choice('loss', values=['binary_crossentropy', 'mean_squared_error', 'huber_loss']),
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter==0:
      model.add(Dense(units=units,activation=activation,input_dim=8))
      model.add(Dropout(hp.Choice('dropot',values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      counter+=1
    else:
      model.add(Dense(units=units,activation=activation))
      model.add(Dropout(hp.Choice('dropot',values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

  model.add(Dense(1,activation=activation))
  model.compile(optimizer=opti,loss=loss,metrics=['accuracy'])

  return model


In [231]:

def build_modelff(hp):
    model = Sequential()

    # Define hyperparameters
    units = hp.Int('neurons', min_value=8, max_value=128, step=8)
    activation = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh'])
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    loss = hp.Choice('loss', values=['binary_crossentropy', 'mean_squared_error', 'huber_loss'])

    # Add input layer with a unique name
    model.add(Dense(units=units, activation=activation, input_dim=8, name='input_layer'))

    # Add hidden layers with unique names
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(units=units, activation=activation, name='hidden_layer_' + str(i)))

    # Add output layer with a unique name
    model.add(Dense(units=1, activation=activation, name='output_layer'))

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    return model


In [250]:
tuner=kt.RandomSearch(build_model4,max_trials=10,objective='accuracy',directory='mydirfinal',project_name='pro6')


In [251]:
tuner.search(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Trial 10 Complete [00h 00m 03s]
accuracy: 0.6791530847549438

Best accuracy So Far: 0.7394136786460876
Total elapsed time: 00h 00m 27s


In [252]:
tuner.get_best_hyperparameters()[0].values

{'neurons': 128,
 'activation': 'tanh',
 'optimizer': 'adam',
 'loss': 'mean_squared_error',
 'num_layers': 2,
 'dropot': 0.3}

In [253]:
model=tuner.get_best_models()[0]

In [255]:
model.fit(x_train,y_train,epochs=100,initial_epoch=11,validation_data=(x_test,y_test))

Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.1614 - accuracy: 0.7606 - val_loss: 0.1585 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1587 - accuracy: 0.7834 - val_loss: 0.1594 - val_accuracy: 0.7857
Epoch 14/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1585 - accuracy: 0.7671 - val_loss: 0.1577 - val_accuracy: 0.7792
Epoch 15/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1600 - accuracy: 0.7655 - val_loss: 0.1558 - val_accuracy: 0.7727
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.1603 - accuracy: 0.7752 - val_loss: 0.1576 - val_accuracy: 0.7922
Epoch 17/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1578 - accuracy: 0.7720 - val_loss: 0.1584 - val_accuracy: 0.7857
Epoch 18/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1583 - accuracy: 0.7801 - val_loss: 0.1580 - val_accuracy: 0.7987
Epoch 